In [ ]:
!pip -q install langchain==0.3.0 langchain-community==0.3.0 langchain-openai==0.2.0 \
                chromadb openai tiktoken

import os
os.environ["OPENAI_API_KEY"] = "dummy"  


In [62]:
import json, os
from langchain.docstore.document import Document

VERT_JSONL_PATH = "/content/drive/MyDrive/VERT.json"

def load_vert_rows(path):
    rows=[]
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try: rows.append(json.loads(line))
                except: pass
    return rows

rows = load_vert_rows(VERT_JSONL_PATH)

def rows_to_docs(rows, max_chars=1200):
    docs=[]
    for r in rows:
        code = (r.get("Code") or "").strip()
        text = f"CODE:\n{code[:max_chars]}"
        docs.append(Document(page_content=text, metadata={}))
    return docs

docs = rows_to_docs(rows)
print(f"Prepared {len(docs)} docs (truncated).")


Prepared 20000 docs (truncated).


In [63]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

emb = OpenAIEmbeddings(model="text-embedding-3-small")

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
docs_split = splitter.split_documents(docs)

vstore = Chroma(collection_name="vert_sva_gen", embedding_function=emb)

BATCH = 128
for i in range(0, len(docs_split), BATCH):
    vstore.add_documents(docs_split[i:i+BATCH])


retriever = vstore.as_retriever(search_kwargs={"k": 2})


Total chunks after split: 20000
Vector store ready.


In [75]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json as pyjson

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    response_format={"type": "json_object"},  # keep JSON out
)

SYS = """You generate strict SystemVerilog Assertions from a requirement and signals.
Return only JSON with fields: assertion, anti_vacuity_cover, notes.

HARD RULES:

========================
INPUTS (variables)
========================
Requirement (text): {requirement}
Signals (JSON map): {signals_json}
  # keys = logical names; values = HDL identifiers, e.g.
  # {{"valid":"vld_i","ready":"rdy_i","data":"payload[7:0]"}}
Clock: {clock}
Reset (active as given): {reset}
Optional params (JSON): {params_json}
  # e.g. {{"ACK_MAX_CYCLES":3,"LAT":4}}

========================
RULES (style & correctness)
========================
- Use clear, descriptive, consistent names for sequences/properties (e.g., data_stable_until_ready).
- If the antecedent is complex/reused, define it as a separate sequence and reuse it.
- Always include explicit clock/reset on every property/cover:
  @(posedge {clock}) disable iff (!{reset})
- Add one short intent comment above each sequence/property.
- Use $stable(x) and $past(x,N) for stability/history. NEVER write "x == $stable(x)".
- Backpressure pattern (valid && !ready): use overlapped implication |-> so enforcement starts the same cycle.
- Use until / until_with to hold until the release event; until_with includes the release cycle.
- The cover must witness both the antecedent and the release/response event (e.g., (valid && !ready) ##[1:$] ready).
- Do not copy RAG exemplars verbatim; adapt to the given requirement and signals.
- Prefer parameters/localparams for time windows instead of magic numbers.
- Output must include:
  1) a named property + `assert property (NAME);`
  2) one `cover property (...)` anti-vacuity witness (never assert a cover).
- Synthesize-safe SVA only; no DPI or tool-specific pragmas.

========================
FEW-SHOT EXAMPLES (for pattern & format — adapt names/signals)
========================
-- Example 1: Backpressure — data must stay stable until READY
// Stall begins when VALID is 1 and READY is 0; keep DATA stable during stall
// and on the READY cycle (release included).
sequence stall_s;
  (valid && !ready);
endsequence

property data_stable_until_ready;
  @(posedge clk) disable iff (!rst_n)
    stall_s |-> $stable(data) until_with (ready);
endproperty
assert property (data_stable_until_ready);

// Anti-vacuity: see a stall that eventually releases.
cover property (@(posedge clk) disable iff (!rst_n)
  stall_s ##[1:$] ready);

// Notes:
// - Overlapped |-> starts stability on the first stall cycle.
// - until_with includes the release (READY) cycle.

-- Example 2: Request must be acknowledged within N cycles
localparam int ACK_MAX_CYCLES = 3; // set via params if provided

// If REQ is high, ACK must arrive within 1..ACK_MAX_CYCLES cycles.
property req_ack_within_n;
  @(posedge clk) disable iff (!rst_n)
    req |-> ##[1:ACK_MAX_CYCLES] ack;
endproperty
assert property (req_ack_within_n);

// Cover: witness a bounded req→ack.
cover property (@(posedge clk) disable iff (!rst_n)
  req ##[1:ACK_MAX_CYCLES] ack);

// Notes:
// - Parameterized window avoids magic numbers.

-- Example 3: Exact latency & value check with $past
localparam int LAT = 4; // set via params if provided

// If START, then exactly LAT cycles later DONE is 1 and RESULT equals the
// input sampled at START.
property result_latency_exact;
  @(posedge clk) disable iff (!rst_n)
    start |-> ##LAT (done && (result == $past(input_data, LAT)));
endproperty
assert property (result_latency_exact);

// Cover: witness a START followed by DONE at LAT.
cover property (@(posedge clk) disable iff (!rst_n)
  start ##LAT done);

// Notes:
// - $past(input_data, LAT) binds RESULT to the sampled input at START.
// - Exact-cycle latency check.

========================
GENERATION STEPS (apply to the current inputs)
========================
1) Parse {requirement}; map logical names to {signals_json}.
2) Choose the closest pattern (hold-until, bounded response, exact latency, etc.).
3) Name sequences/properties descriptively; parameterize any counts from {params_json} if present.
4) Generate the three sections in the exact format.
5) Quality gate (must all pass):
   - Has @(posedge {clock}) disable iff (!{reset})
   - Uses $stable/$past correctly (no "x == $stable(x)")
   - Backpressure uses |-> if applicable
   - No magic numbers (use params/localparams)
   - Includes assert property (NAME);
   - Includes a cover witnessing antecedent then release/response
"""

USR = """CLOCK: {clock}
RESET: {reset}
SIGNALS (pretty): {signals}
REQUIREMENT: {requirement}

RETRIEVED EXEMPLARS:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("system", SYS), ("user", USR)])
chain = prompt | llm

def generate_sva(requirement: str,
                 signals: dict,
                 clock: str = "clk",
                 reset: str = "rst_n",
                 params: dict | None = None,
                 k: int = 30):
    # Retrieve context (assumes `retriever` is defined elsewhere)
    query = requirement + " " + " ".join(signals.keys())
    hits = retriever.invoke(query)
    context = "\n\n---\n".join([h.page_content for h in hits])

    # Prepare variables for the prompt
    signallist = ", ".join(f"{k}={v}" for k, v in signals.items())
    signals_json = pyjson.dumps(signals, ensure_ascii=False)
    params_json = pyjson.dumps(params or {}, ensure_ascii=False)

    res = chain.invoke({
        "clock": clock,
        "reset": reset,
        "signals": signallist,       # used by USR
        "signals_json": signals_json, # used by SYS
        "params_json": params_json,   # used by SYS
        "requirement": requirement,
        "context": context,
    })

    out = pyjson.loads(res.content)
    assert "assertion" in out and "anti_vacuity_cover" in out and "notes" in out
    assert "assert property" in out["assertion"]
    assert "cover property" in out["anti_vacuity_cover"]
    return out


/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if (await self.run_code(code, result,  async_=asy)):


In [65]:
def display_sva(out: dict):
    """Pretty-print the SVA generator output dict."""
    a = (out.get("assertion") or "").strip()
    c = (out.get("anti_vacuity_cover") or "").strip()
    n = (out.get("notes") or "").strip()
    line = "-" * 60

    print(f"{line}\nASSERTION\n{line}\n{a}\n")
    print(f"{line}\nANTI-VACUITY COVER\n{line}\n{c}\n")
    if n:
        print(f"{line}\nNOTES\n{line}\n{n}\n")




In [76]:
resp = generate_sva(
    requirement="When valid is high and ready is low, data must remain stable until ready goes high.",
    signals={"valid":"valid","ready":"ready","data":"data[7:0]"},
    clock="clk", reset="rst_n"
)
display_sva(resp)


------------------------------------------------------------
ASSERTION
------------------------------------------------------------
property data_stable_until_ready; @(posedge clk) disable iff (!rst_n) (valid && !ready) |-> $stable(data) until_with (ready); assert property (data_stable_until_ready);

------------------------------------------------------------
ANTI-VACUITY COVER
------------------------------------------------------------
cover property (@(posedge clk) disable iff (!rst_n) (valid && !ready) ##[1:$] ready);

------------------------------------------------------------
NOTES
------------------------------------------------------------
- Overlapped |-> starts stability on the first stall cycle.
- until_with includes the release (READY) cycle.

